In [99]:
import torch.nn as nn
import pytorch_lightning as pl
import torch

In [100]:
from typing import Any


class SelfAttention(nn.Module):
    """
    Self attention layer
    """

    def __init__(self, embed_size, heads):
        
        super().__init__()
        self.embed_size = embed_size
        self.heads = heads

        self.head_dim = embed_size // heads

        assert (self.head_dim * heads == embed_size), "Embed size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0] # batch size
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        #split embedding into self.heads pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values) # (N, value_len, heads, head_dim)
        keys = self.keys(keys)
        query = self.queries(query)

        energy = torch.einsum("nqhd,nkhd->nhqk", [query, keys])
        # query shape: (N, query_len, heads, head_dim)
        # keys shape: (N, key_len, heads, head_dim)
        # energy shape: (N, heads, query_len, key_len)

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        
        attention = torch.softmax(energy / (self.embed_size ** (1/2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values])  

        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)    
        # out after matrix multiply: (N, query_len, heads, head_dim)

        out = out.reshape(N, query_len, self.heads * self.head_dim)
        # out shape: (N, query_len, embed_size)

        out = self.fc_out(out)

        return out


In [101]:
nn.Sequential(
            nn.Linear(10, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
        )

Sequential(
  (0): Linear(in_features=10, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=64, bias=True)
)

In [102]:
class TransformerBlock(nn.Module):

    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super().__init__()
        self.attention = SelfAttention(embed_size, heads)

        self.norm1 = nn.LayerNorm(embed_size)

        self.norm2 = nn.LayerNorm(embed_size)

        print(embed_size, forward_expansion * embed_size)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size)
        )

        self.dropout = nn.Dropout(dropout)


    
    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        skip1 = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(skip1)
        skip2 = self.dropout(self.norm2(forward + skip1)) 

        return skip2
    

In [103]:
class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embed_size, num_layers, heads, device, forward_expansion, dropout, max_length):
        super().__init__()
        self.embed_size = embed_size
        self.device = device
        
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.positional_embedding = nn.Embedding(max_length, embed_size) # it is not the positional encoidding of the paper, but it is the same idea
        # the other uses the sinusoidal positional encoding

        self.layers = nn.ModuleList([
            TransformerBlock(
                embed_size, heads, dropout=dropout, forward_expansion=forward_expansion
            )
            
        ])

        self.dropout = nn.Dropout(dropout)

    
    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        out = self.dropout(self.word_embedding(x) + self.positional_embedding(positions))

        for layer in self.layers:
            out = layer(out, out, out, mask)
        
        return out
    


In [104]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super().__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.transformer_block = TransformerBlock(embed_size, heads, dropout, forward_expansion)
        self.dropout = nn.Dropout(dropout)

    
    def forward(self, x, value, key,  src_mask,  trg_mask):
        attention = self.attention(x, x, x, trg_mask) # masked multi-head attention
        query = self.dropout(self.norm(attention + x)) # additive residual connection and normalization
        out = self.transformer_block(value, key, query, src_mask)
        return out


In [105]:
class Decoder(nn.Module):

    def __init__(self,
                 trg_vocab_size,
                 embed_size,
                 num_layers,
                 heads,
                 forward_expansion,
                 dropout,
                 device,
                 max_length):
        super().__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size,heads, dropout=dropout, forward_expansion=forward_expansion, device=device) 
            
            for _ in range(num_layers)]
        )

    
    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.word_embedding(x) + self.position_embedding(positions)

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)
        
        return x


In [108]:
from typing import Any


class Transformer(pl.LightningModule):
    def __init__(self, 
                 src_vocab_size,
                 trg_vocab_size,
                 src_pad_idx,
                 trg_pad_idx,
                 embed_size=256,
                 num_layers=6,
                 forward_expansion=4,
                 heads=8,
                 dropout=0.1,
                 device="cuda",
                 max_length=100):
        super().__init__()
        
        self.encoder = Encoder(src_vocab_size=src_vocab_size,
                                 embed_size=embed_size,
                                 num_layers=num_layers,
                                 heads=heads,
                                 device=device,
                                 forward_expansion=forward_expansion,
                                 dropout=dropout,
                                 max_length=max_length)
        self.decoder = Decoder(trg_vocab_size=trg_vocab_size,
                                    embed_size=embed_size,
                                    num_layers=num_layers,
                                    heads=heads,
                                    device=device,
                                    forward_expansion=forward_expansion,
                                    dropout=dropout,
                                    max_length=max_length)

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx


    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(N, 1, trg_len, trg_len)
        # (N, 1, trg_len, trg_len)
        return trg_mask.to(self.device)
    
    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, trg_mask, src_mask)
        return out



In [123]:

# test the transformer model

import torch

src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 1, 0]])
tgt = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 1, 0]])

#model = Transformer(src_vocab_size=11, trg_vocab_size=11, src_pad_idx=0, trg_pad_idx=0, device="cpu")
out = model(src, tgt)


In [124]:
out

tensor([[[-0.0000, -0.3477,  0.0623,  ..., -0.0971, -0.3647,  0.2401],
         [ 0.3695, -0.0000, -0.1089,  ..., -1.2637, -0.4216, -0.0414],
         [-0.4604,  3.3015,  0.4987,  ..., -0.0142, -0.2659,  0.3428],
         ...,
         [ 3.7196,  4.1042,  0.2719,  ..., -0.3835, -0.0000, -0.1760],
         [-0.6158,  0.2001, -4.2879,  ...,  0.0981, -0.1111, -0.3317],
         [-0.6386, -0.0670,  0.4058,  ...,  0.0893, -0.0428, -0.3221]]],
       grad_fn=<MulBackward0>)

In [126]:
out.shape

torch.Size([1, 13, 256])